Рогович Татьяна

# Анализ данных  в Python (политология)
## 1. Разбираем кейс с парсингом email в плохо собранных данных
##### Занятия 9-10 | 16.10.2019

In [2]:
import pandas as pd
import re

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/rogovich/2019-2020_PolSci_Data_Analysis_in_Python/master/5week_Pandas_Exercises/data_contacts.csv', index_col=0)
data

,01. Имя,8. Личные контакты
0,Елизавета,NaN
1,Екатерина,mail2@edu.hse.ru
2,Рената,+7915472999 kdfbnd-1998@mail.ru
3,Полина,NaN
4,Кирилл,wev.evwunev@gmail.com
5,Анна,8909915999\nbunevls@yandex.ru
6,Михаил,NaN
7,Юлия,hvbiuqok@mail.ru\n8-(962)-949-99-99
8,Анна,89272169999// mail9@edu.hse.ru
9,Ольга,+79099619999\njeovsv.omwivweb@hotmail.com


К сожалению, уже в процессе обсуждения мы обнаружили, что наш код не учитывает почтовые домены с двумя точками (edu.hse.ru). Но это, например, можно обойти функцией, которая обрабатывает нужные случаи.

Рассмотрим работу регулярки на данном примере: 'iawpghnube1206@gmail.com\r\n+79151489999 (telegram @vasiiesal)'

r - начало регулярки (спецсимвол), а тело самого решглярного выражения внутри ''
\b - это границы "слова": делает по факту разбивку на "слова", по пробелу, символам каретки, знаки переноса строки и т.д.

Например, \b разобьет на следующие подстроки:
    * iawpghnube1206
    * @
    * gmail
    * .
    * com
    * \
    * r
    * \
    * n
    * +
    * 7
    * (
    * @
    * v
    * )



In [4]:
x = 'iawpghnube1206@gmail.com\r\n+79151489999 (telegram @vasiiesal)'
re.findall(r'\b', x) # строки в выводе пустые, потому что не указали какой паттерн искать

['', '', '', '', '', '', '', '', '', '', '', '']

Далее идет блок в [ ], в котором как раз определяем паттерн, который ищем, в нашем случае это \w\d\S, где \w Любая цифра или буква, \S любой непробельный символ. НО это только один символ, попадающий в эту маску. Для того чтобы получить неограниченную длинной последовательность, мы добави + к квадратным скобкам.

Получим такие вот подстроки:
    * iawpghnube1206@gmail.com\r\n+79151489999
    * telegram
    * vasiiesal)

In [5]:
re.findall(r'\b[\w\S]+', x)

['iawpghnube1206@gmail.com', '79151489999', 'telegram', 'vasiiesal)']

@ показывает нам, что далее нужно найти 1 символ @. Что даст следующие подстроки:
    * iawpghnube1206@
    * vasiiesal)


In [6]:
re.findall(r'\b[\w\S]+@', x)

['iawpghnube1206@']

Т.к. мы знаем что после @ всегда идет домен, то далее мы ищем только буквы \w, точнее последовательность из букв \w+. Получим уже одну строку:
    * iawpghnube1206@gmail


In [7]:
re.findall(r'\b[\w\S]+@\w+', x)

['iawpghnube1206@gmail']

А далее ждет доменная зона, которая всегда начинается с точки, поэтому и ищем 1 символ ., а за ним любую буквенную последовательность \w+, что и даст нам искомые почты
    * iawpghnube1206@gmail.com
    
Детально посмотреть работу регулярного выражения можно на этом сайте. Там каждое значые объясняется и выделяется в искомой строке. Очень удобно.
https://regex101.com/

In [8]:
re.findall(r'\b[\w\d\S]+@\w+.\w+', x)

['iawpghnube1206@gmail.com']

In [9]:
for i in data['8. Личные контакты']:
    mails = re.findall(r'\b[\w\d\S]+@\w+.\w+', str(i))
    print(mails)

[]
['mail2@edu.hse']
['kdfbnd-1998@mail.ru']
[]
['wev.evwunev@gmail.com']
['bunevls@yandex.ru']
[]
['hvbiuqok@mail.ru']
['mail9@edu.hse']
['jeovsv.omwivweb@hotmail.com']
['ogjgei.owenvejkr@yandex.ru']
['pdfko_588@mail.ru']
['lwegmbek.hse@gmail.com']
['Korbjtein424@gmail.com']
['Lounwbhr@gmail.com']
['ktom.quwioergkbj@gmail.com']
['gog.otmn@mail.ru']
['iawpghnube1206@gmail.com']
['kptkg.oewmibri@inbox.ru', 'Poenb89252778480@yandex.ru']
['mail20@edu.hse']
['ieminbbr98@gmail.com']
['owjvnbmjregs.98@mail.ru']
['zxcvbhjyt@yandex.ru']
['Zotovasofya1999@yandex.ru']
['kgquiduhr@gmail.com']
['mail27@edu.hse', 'tmbtjneiwmecvt@gmail.com']
['invjrnvox@mail.ru']
['mntubnjr@gmail.com', 'mail28@edu.hse']


In [71]:
data['email'] = None
data['email2'] = None

In [72]:
for i in range(len(data)):
    mails = re.findall(r'\b[\w\d\S]+@\w+.\w+', str(data['8. Личные контакты'][i]))
    try:
        data['email'][i] = mails[0]
        data['email2'][i] = mails[1]
    
    except:
        continue

[]
['mail2@edu.hse']
['kdfbnd-1998@mail.ru']
[]
['wev.evwunev@gmail.com']
['bunevls@yandex.ru']
[]
['hvbiuqok@mail.ru']
['mail9@edu.hse']
['jeovsv.omwivweb@hotmail.com']
['ogjgei.owenvejkr@yandex.ru']
['pdfko_588@mail.ru']
['lwegmbek.hse@gmail.com']
['Korbjtein424@gmail.com']
['Lounwbhr@gmail.com']
['ktom.quwioergkbj@gmail.com']
['gog.otmn@mail.ru']
['iawpghnube1206@gmail.com']
['kptkg.oewmibri@inbox.ru', 'Poenb89252778480@yandex.ru']
['mail20@edu.hse']
['ieminbbr98@gmail.com']
['owjvnbmjregs.98@mail.ru']
['zxcvbhjyt@yandex.ru']
['Zotovasofya1999@yandex.ru']
['kgquiduhr@gmail.com']
['mail27@edu.hse', 'tmbtjneiwmecvt@gmail.com']
['invjrnvox@mail.ru']
['mntubnjr@gmail.com', 'mail28@edu.hse']


In [10]:
data

,01. Имя,8. Личные контакты
0,Елизавета,NaN
1,Екатерина,mail2@edu.hse.ru
2,Рената,+7915472999 kdfbnd-1998@mail.ru
3,Полина,NaN
4,Кирилл,wev.evwunev@gmail.com
5,Анна,8909915999\r\nbunevls@yandex.ru
6,Михаил,NaN
7,Юлия,hvbiuqok@mail.ru\r\n8-(962)-949-99-99
8,Анна,89272169999// mail9@edu.hse.ru
9,Ольга,+79099619999\r\njeovsv.omwivweb@hotmail.com


Тьюториал по регуляркам, если забыли их
https://tproger.ru/translations/regular-expression-python/